In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_rms(x, axis=None):
    return np.sqrt(np.mean(x ** 2, axis=axis))

def calc_sma_for_window(data):
    return np.sum(data) / len(data)


def calc_sma_adv_for_window(data):
    return np.sum(data - np.mean(data) / len(data))


def calc_absolutes_for_list(list):
    return ([abs(i) for i in list])

def get_sma(data): 
    sma_sim = calc_sma_for_window(data)
    sma_adv = calc_sma_adv_for_window(data)

    sma_sim_abs = calc_sma_for_window(calc_absolutes_for_list(data))
    sma_adv_abs = calc_sma_adv_for_window(calc_absolutes_for_list(data))

    return sma_sim, sma_adv, sma_sim_abs, sma_adv_abs

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Histogram_equalization

In [8]:
def histogram_equalize(im):#https://www.researchgate.net/publication/281118372_NumPy_SciPy_Recipes_for_Image_Processing_Intensity_Normalization_and_Histogram_Equalization
    data = im.copy().flatten()
    hist, bins = np.histogram(data, 256, density=True)
    cdf = hist.cumsum()#normalized cumulative histogram H using NumPy’s inbuilt function cumsum
    cdf = 255*cdf/cdf[-1]  #intensity transformation
    img_eq = np.interp(data, bins[:-1], cdf)
    return img_eq.reshape(im.shape)

In [9]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [10]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [11]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
   
    for subject in range(1,18):
        file_path = dataset_path + '\subject{0}_ideal.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [12]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

# HISTOGRAM EQUALIZATION on features

In [13]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
    features = histogram_equalize(X)  # HISTOGRAM EQUALIZATION
    X = features
        
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    print(accuracy)
    return accuracy

In [14]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [15]:
dataset_csv_path = r"D:\projec\proj\data"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [16]:
ALL.head(10)

,group,0,1,2,3,4,5,6,7,8,...,233,234,235,236,237,238,239,240,241,242
0,1,-0.317008,0.074480,0.074480,-13.572,13.123,-5563.499014,7.228819,51.0,73.0,...,3.433057,0.096471,0.409673,0.065249,0.081963,0.060492,0.428163,0.104005,0.407340,1
1,1,-0.320368,0.068405,0.068405,-13.572,13.123,-5622.463930,7.228819,54.0,82.0,...,2.955452,0.095075,0.413741,0.068959,0.064359,0.058963,0.406512,0.092348,0.442178,1
2,1,-0.323965,0.068771,0.068771,-13.572,13.123,-5685.586551,7.228819,55.0,84.0,...,2.090189,0.071682,0.437328,0.062415,0.059816,0.050159,0.424356,0.094437,0.431047,1
3,1,-0.336631,0.071631,0.071631,-13.547,10.304,-5907.879152,7.228819,27.0,90.0,...,2.260539,0.066683,0.459112,0.052358,0.067397,0.045673,0.462147,0.108078,0.384102,1
4,1,-0.348977,0.073038,0.073038,-13.547,12.127,-6124.541712,7.228819,22.0,95.0,...,3.138315,0.075594,0.467472,0.047577,0.067944,0.051054,0.470697,0.113735,0.364515,1
5,1,-0.371705,0.086086,0.086086,-16.693,13.038,-6523.425930,7.228819,37.0,90.0,...,3.324015,0.098932,0.452130,0.049836,0.075622,0.054213,0.458554,0.109609,0.377625,1
6,1,-0.397238,0.088222,0.088222,-16.693,13.038,-6971.534233,7.228819,42.0,76.0,...,3.237408,0.099800,0.440110,0.051009,0.077333,0.055490,0.446425,0.105752,0.392333,1
7,1,-0.435878,0.082128,0.082128,-17.479,18.240,-7649.662100,7.228819,47.0,54.0,...,3.112343,0.092486,0.455538,0.052813,0.058612,0.055669,0.429769,0.098172,0.416389,1
8,1,-0.467565,0.093603,0.093603,-18.348,18.240,-8205.770461,7.228819,51.0,56.0,...,2.196212,0.069423,0.463414,0.048845,0.073784,0.048486,0.451738,0.104297,0.395479,1
9,1,-0.503556,0.096744,0.096744,-18.348,18.240,-8837.401092,7.228819,53.0,58.0,...,3.605017,0.084708,0.479525,0.042276,0.069691,0.051493,0.466764,0.110999,0.370745,1


In [17]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.9692307692307692
Model- 2  -  f1 score:  0.985900599224533
Model- 3  -  f1 score:  0.9708
Model- 4  -  f1 score:  0.9747590637907297
Model- 5  -  f1 score:  0.9572413793103448
Model- 6  -  f1 score:  0.96
Model- 7  -  f1 score:  0.997229916897507
Model- 8  -  f1 score:  0.9543010752688172
Model- 9  -  f1 score:  0.9654700854700855
Model- 10  -  f1 score:  0.9836121543188802
Model- 11  -  f1 score:  0.974375
Model- 12  -  f1 score:  0.9717336683417085
Model- 13  -  f1 score:  0.9410104431272932
Model- 14  -  f1 score:  0.9602252727912707
Model- 15  -  f1 score:  0.9731485491554785
Model- 16  -  f1 score:  0.9090577063550036
Model- 17  -  f1 score:  0.9558767592240395
0.9649395554415565


0.9649395554415565

In [18]:
d ={'ALL': [ALL_accuracy]}

In [19]:
df_accuracy = pd.DataFrame(data=d)

In [20]:
df_accuracy

,ALL
0,0.96494
